In [1]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np

from match_elements.GUI_pair import GUIPair
from GUI import GUI
from Operation import Operation

In [2]:
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

In [2]:
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')

[2022/02/27 12:07:01] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/02/27 12:07:01] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/02/27 12:07:01] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR
Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/det/en/en_ppocr_mobile_v2.0_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_s

In [3]:
gui = GUI('data/input/0.jpg')
# gui.element_detection(True, True, True, paddle_cor=paddle_ocr)
gui.load_detection_result()
# gui.show_detection_result()

In [11]:
testing_data_root = '/home/ml/Data/visual testing/experiments/humanoid-output/adguard-1'

steps_file = pjoin(testing_data_root, 'output.json')
steps = json.load(open(steps_file, 'r'))

operations = []
for step in steps:
    op = Operation(pjoin(testing_data_root, step['tag'] + '.jpg'), step['event_type'], step['bounds'])
    op.shrink_target_element_img()
    op.resize(gui.detection_resize_width, gui.detection_resize_height)        
    op.detect_text(paddle_ocr)
    operations.append(op)
    op.show_target_ele()

[2022/02/27 12:10:06] root DEBUG: dt_boxes num : 1, elapse : 0.0163729190826416
[2022/02/27 12:10:06] root DEBUG: cls num  : 1, elapse : 0.006376504898071289
[2022/02/27 12:10:06] root DEBUG: rec_res num  : 1, elapse : 0.004773139953613281
[Text Detection Completed in 0.031 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/27 12:10:08] root DEBUG: dt_boxes num : 3, elapse : 0.11468625068664551
[2022/02/27 12:10:08] root DEBUG: cls num  : 3, elapse : 0.01301121711730957
[2022/02/27 12:10:08] root DEBUG: rec_res num  : 3, elapse : 0.03494906425476074
[Text Detection Completed in 0.180 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/27 12:10:08] root DEBUG: dt_boxes num : 1, elapse : 0.04717659950256348
[2022/02/27 12:10:08] root DEBUG: cls num  : 1, elapse : 0.0062563419342041016
[2022/02/27 12:10:08] root DEBUG: rec_res num  : 1, elapse : 0.006976604461669922
[Text Detection Completed in 0.067 s] Input: data/operation/target.

[2022/02/27 12:10:21] root DEBUG: cls num  : 0, elapse : 0
[2022/02/27 12:10:21] root DEBUG: rec_res num  : 0, elapse : 9.5367431640625e-07
[Text Detection Completed in 0.023 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/27 12:10:22] root DEBUG: dt_boxes num : 0, elapse : 0.015033245086669922
[2022/02/27 12:10:22] root DEBUG: cls num  : 0, elapse : 0
[2022/02/27 12:10:22] root DEBUG: rec_res num  : 0, elapse : 1.430511474609375e-06
[Text Detection Completed in 0.018 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/27 12:10:23] root DEBUG: dt_boxes num : 3, elapse : 0.11504626274108887
[2022/02/27 12:10:23] root DEBUG: cls num  : 3, elapse : 0.011212587356567383
[2022/02/27 12:10:23] root DEBUG: rec_res num  : 3, elapse : 0.03509855270385742
[Text Detection Completed in 0.177 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/27 12:10:23] root DEBUG: dt_boxes num : 1, elapse : 0.046614170074462

In [7]:
gui.match_elements(operations[0].target_element_img, resnet_model=resnet_model)

0.9386408
0.563463
0.38649696
0.5936606
0.51987505


In [12]:
operations[1].show_target_ele()

In [13]:
[t.content for t in operations[1].target_element_text]

['ADGUARD v3.3 beta 3.1', 'Trial license', 'Until February 24.2022']

In [9]:
# ***************************
# *** Streaming Detection ***
# ***************************

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)

while(1):
    ret, frame = cap.read()
    
    cv2.imshow('camera', frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('s'):
        cv2.imwrite('data/output/temp.jpg', frame)
        gui = GUI('data/output/temp.jpg')
        gui.element_detection(True, True, True, paddle_cor=paddle_ocr)
        cv2.imshow('result', gui.det_result_imgs['merge'])
        print('===================\n')
cap.release()
cv2.destroyAllWindows()

[2022/02/26 13:25:22] root DEBUG: dt_boxes num : 0, elapse : 0.14470863342285156
[2022/02/26 13:25:22] root DEBUG: cls num  : 0, elapse : 0
[2022/02/26 13:25:22] root DEBUG: rec_res num  : 0, elapse : 1.1920928955078125e-06
[Text Detection Completed in 0.161 s] Input: data/output/temp.jpg Output: data/output/ocr/temp.json
[Compo Detection Completed in 1.063 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg

